In [1]:
import os
import sys
import glob
import numpy as np
import pickle
from tqdm import tqdm

import pandas as pd

from sklearn.metrics import precision_score, roc_auc_score

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys

sys.path.append('../pytorch')

In [3]:
import imp

import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

import torch.optim as optim

import utils.data_processor as data_processor
imp.reload(data_processor)
from utils.data_processor import *

import utils.visualization_tools as visualization_tools
imp.reload(visualization_tools)
from utils.visualization_tools import *

import utils.metrics as metrics
imp.reload(metrics)
from utils.metrics import *

Loading dataset for grey matter segmentation

In [4]:
with open("../pytorch/data/BrainData/trainval_data_exp_grey.pkl", "rb") as f:
    pcs_train,labels_train, _ = pickle.load(f)
    
with open("../pytorch/data/BrainData/test_data_exp_grey.pkl", "rb") as f:
    pcs_test,labels_test, _ = pickle.load(f)

In [5]:
from dataset import PCDataset

uploading model and training pipeline

In [6]:
from DGCNN import DGCNN_semseg
from train_eval import train_val, val_step

In [7]:
device="cuda:0"

In [8]:
path_to_save=f"models/grey_noise.pth"

In [9]:
#add argument std to train_val and uncomment if you want to train the models with the noise 

std = 0.005

In [10]:
n_epochs = 12
num_point_cloud = 20000
batch_size = 1

In [11]:
train_data = PCDataset(pcs_train,labels_train, num_point_cloud)
test_data = PCDataset(pcs_test,labels_test, num_point_cloud)

In [12]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [13]:
result_dict = {"BCE": [], "IoU": [], "ROC-AUC": [], "Dice": []}

In [14]:
point_cloud = next(iter(trainloader))
target = point_cloud[1].float().detach().cpu().numpy().reshape(-1)

In [15]:
model = DGCNN_semseg(k=20, emd_dims=512, dropout=0.5)
model.to(device)

opt = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)
criterion = nn.BCEWithLogitsLoss()

metrics_dict = train_val(model, opt, criterion, trainloader, valloader,
                         n_epochs=n_epochs, is_train=True, val_step_func=val_step, std=std)


torch.save(model.state_dict(), path_to_save)

Epoch = 0, Train loss: 0.318, Test loss: 0.382
Train ROC-AUC: 0.931, Test ROC-AUC: 0.936
Train IoU: 0.714, Test IoU: 0.608
Train Dice: 0.828, Test Dice: 0.721
Epoch = 1, Train loss: 0.246, Test loss: 0.322
Train ROC-AUC: 0.96, Test ROC-AUC: 0.943
Train IoU: 0.767, Test IoU: 0.677
Train Dice: 0.868, Test Dice: 0.792
Epoch = 2, Train loss: 0.235, Test loss: 0.286
Train ROC-AUC: 0.963, Test ROC-AUC: 0.947
Train IoU: 0.773, Test IoU: 0.719
Train Dice: 0.872, Test Dice: 0.83
Epoch = 3, Train loss: 0.229, Test loss: 0.276
Train ROC-AUC: 0.965, Test ROC-AUC: 0.949
Train IoU: 0.777, Test IoU: 0.73
Train Dice: 0.874, Test Dice: 0.838
Epoch = 4, Train loss: 0.226, Test loss: 0.272
Train ROC-AUC: 0.966, Test ROC-AUC: 0.95
Train IoU: 0.779, Test IoU: 0.734
Train Dice: 0.876, Test Dice: 0.841
Epoch = 5, Train loss: 0.223, Test loss: 0.271
Train ROC-AUC: 0.967, Test ROC-AUC: 0.951
Train IoU: 0.781, Test IoU: 0.736
Train Dice: 0.877, Test Dice: 0.842
Epoch = 6, Train loss: 0.221, Test loss: 0.27
Trai